#### Лабораторная работа 6. Задачи квадратичного программирования
Реализовать решение задачи квадратичного программирования.

$----------------------------------------------$

In [256]:
import numpy as np

<u>Шаг 0.</u> Определение исходных данных задачи

In [257]:
D = np.array([[2, 1, 1, 0], [1, 1, 0, 0], [1, 0, 1, 0], [0, 0, 0, 0]])
A = np.array([[1, 0, 2, 1], [0, 1, -1, 2]])
c = np.array([[-8], [-6], [-4], [-6]])
x_t = np.array([[2], [3], [0], [0]])
j_b = [0, 1]
j_b_extd = [0, 1]

<u>Шаг 1.</u> Находим векторы $c(x)$, $u(x)$ и $\Delta(x)$ по формулам:


$c'(x) = c' + x' \cdot D$


$u'(x) = -c_{b}'(x) \cdot A_{b}^{-1}$


$\Delta'(x) = u'(x) \cdot A + c'(x)$


In [258]:
c_x = np.add(c, D.dot(x_t))

idx = np.array(j_b)
c_b = c_x[idx]
A_b = A[:, idx]
A_b_inv = np.linalg.inv(A_b)
u_x = A_b_inv.dot(-1*c_b)

delta_x = np.add(np.transpose(u_x)[0].dot(A), np.transpose(c_x)[0])

print("c_x = ", c_x, "\nu_x = ", u_x,"\ndelta_x = ", delta_x)

c_x =  [[-1]
 [-1]
 [-2]
 [-6]] 
u_x =  [[1.]
 [1.]] 
delta_x =  [ 0.  0. -1. -3.]


<u>Шаг 2.</u> Проверяем условие оптимальности текущего правильного опорного плана. Если все компоненты вектора $\Delta(x)$ неотрицательные, то метод завершает свою работу и текущий правильный опорный план является оптимальным. В противном случае переходим на следующий шаг.

In [259]:
if np.all(delta_x >= 0):
    j_b = [i + 1 for i in j_b]
    j_b_extd = [i + 1 for i in j_b_extd]
    print("ОТВЕТ:")
    print("x =", np.transpose(x_t))
    print("j_b =", j_b)
    print("j_b_extd =", j_b_extd)
    break
else:
    print("План неоптимальный")


План неоптимальный


<u>Шаг 3.</u> Выбираем отрицательную компоненту плана $\Delta(x)$ и индекс выбранной компоненты запоминаем в переменной $j_{0}$

In [260]:
j_0 = np.where(delta_x < 0)[0][0]
j_0

2

<u>Шаг 4.</u> По множеству $J_{b_{*}}$ и $j_{0}$ найдём вектор $l' = (l_{1}, l_{2}, l_{3}, l_{4})$

Компоненты этого вектора делятся на два класса:

$l_{b_{*}} = \begin{pmatrix}
l_1 \\
l_2 \\
\end{pmatrix}$

$\bar{l_{b_{*}}} = \begin{pmatrix}
l_3 \\
l_4 \\
\end{pmatrix}$

(1) Сперва находим вектор $\bar{l_{b_{*}}}$ по следующему правилу: $j_{0}$ = 1, значения всех остальных компонент $\bar{l_{b_{*}}}$ полагаем равными 0.


In [261]:
l = np.zeros(delta_x.size)
l[j_0] = -next(
    (item for item in delta_x if item < 0),
    None)

(2) Для того, чтобы найти компоненты вектора $l_{b_{*}}$ мы составляем матрицу H 

$H = \begin{pmatrix}
D^{*} & A_{b^{*}}'\\
A_{b^{*}} & 0\\
\end{pmatrix}$


и обращаем её

In [262]:
D_sub = D[np.ix_(j_b, j_b)]
H_1 = np.concatenate((D_sub, np.transpose(A_b)), axis=1)
H_2 = np.concatenate((A_b, np.zeros(A_b.shape)), axis=1)
H = np.concatenate((H_1, H_2), axis=0)
H_inv = np.linalg.inv(H)
H

array([[2., 1., 1., 0.],
       [1., 1., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

(3) Далее строим вектор $b^{*}$ Он состоит из двух частей. Сперва идут элементы столбца матрицы D с индексом $j_0$, стоящие в строках с индексами из множества $J_{b^{*}}$. Далее идут элементы $j_0$-го столбца матрицы A.

In [263]:
d_j0 = D[:, j_0]
b = []
for item in j_b_extd:
    b.append(d_j0[item - 1])
a_j0 = A[:, j_0]
for item in a_j0:
    b.append(item)
b

[0, 1, 2, -1]

(4) Находим вектор x по следующей формуле

$\hat x = −H^{−1} \cdot b^{∗}$

In [264]:
tmp = np.dot(-1, H_inv)
x_hat = np.dot(tmp, b)

x_hat

array([-2.,  1.,  3.,  0.])

(5) Возьмем недостающее число компанент для $l_{b_{*}}$ из $\hat x$ (берем цельным блоком из начала вектора)

In [265]:
for i in range(len(j_b_extd)):
    l[j_b_extd[i]] = x_hat[i]
    
l

array([-2.,  1.,  1.,  0.])

<u>Шаг 5.</u> Для каждого индекса $j \in J_{b^{*}}$ найдём величину $\theta_{j}$, а также вычислим величину $\theta_{j_{0}}$.

(1) Сперва вычислим $\delta$

$\delta = l' \cdot D \cdot l$

In [266]:
lil_delta = np.transpose(l) * D * l
lil_delta = np.linalg.matrix_rank(lil_delta)
lil_delta

2

(2) Теперь найдем $\theta_{j_{0}}$ по следующему правилу:

$\theta_{j_{0}} = \begin{cases}
\infty, \space если \space \delta=0\\
\frac{|{\Delta_{j_{0}}(x)}|}{\delta}, \space если \space \delta>0
\end{cases}$

In [267]:
theta_j_0 = 0
if lil_delta > 0:
    theta_j_0 = abs(delta_x[j_0]) / lil_delta
elif lil_delta == 0:
    theta_j_0 = np.inf

theta_j_0

0.5

(3) Далее найдем $\theta_{j}$ для каждого $j \in J_{b^{*}}$

$\theta_{j} = \begin{cases}
-\frac{x_{j}}{l_{j}}, \space если \space l_{j}<0\\
\infty, \space если \space l_{j}\geq 0
\end{cases}$

In [268]:

theta_li = [theta_j_0]
index_li = [j_0]
for j in j_b_extd:
    if l[j] < 0:
        theta_li.append((-1)*(float(x_t[j])/float(l[j])))
    elif l[j] >= 0:
        theta_li.append(np.inf)
    index_li.append(j)
    
print(theta_li)
print(index_li)

[0.5, 1.0, inf]
[2, 0, 1]


(4) Теперь найдем минимум $\theta_{0}$

In [269]:
theta_0_index = 0
theta_0 = theta_li[0]
for i in range(len(theta_li)):
    if theta_0 > theta_li[i]:
        theta_0 = theta_li[i]
        theta_0_index = i

if (theta_0 == np.inf):
    print("Целевая функция задачи не ограничена снизу на множестве допустимых планом")
else:
    j_str = index_li[theta_0_index]
j_str

2

<u>Шаг 6.</u> Обновим допустимый план 

(1) $x = x + \theta_0 \cdot l$

In [270]:
x_t = np.transpose(np.add(np.transpose(x_t), l.dot(theta_0)))
x_t

array([[1. ],
       [3.5],
       [0.5],
       [0. ]])

(2) Обновим теперь опору ограничений $J_b$ и расширенную опору ограничений $J_{b_*}$ по следующему правилу:

1. Если $j_* = j_0$, то $J_b$ не меняем, а в $J_{b^*}$ добавляем $j_*$
2. Если $j_* \in J_b$, то $J_b$ не меняем, а из $J_{b^*}$ удаляем $j_*$
3. Если $j_* \in J_b$ (индекс $j_*$ идёт $s$-м по счёту в $J_b$) и существует индекс $j_+ \in J_{b^*} \backslash J_b$ такой, что $s$-я компонента вектора $(A^{−1}_{b} · A_{j_+} )$ не равна 0, то в $J_b$ заменяем индекс $j_*$ на индекс $j_+$, а из $J_{b_*}$ удаляем индекс $j_*$.
4. Если $j_* \in J_b$ (индекс $j_*$ идёт $s$-м по счёту в $J_b$) и ($J_b = J_{b_*}$ или для любого индекса $j_+ \in J_{b_*} \backslash J_b$ выполняется условие: $s$-ая компонента вектора $(A^{−1}_{b} · A_{j_+} )$ равна 0), то и в $J_b$, и в $J_{b_*}$ заменяем индекс $j_*$ на индекс $j_0$.


In [271]:
if j_str == j_0:
    j_b_extd.append(j_str)
elif (j_str in j_b_extd) and (j_str not in j_b):
    j_b_extd.delete(j_str)
elif j_str in j_b:
    s_list = np.where(j_b == j_str)
    j_plus_list = np.where(j_b_extd != j_b)
    break_flag = False
    for s in s_list:
        for j_plus in j_plus_list:
            if np.linalg.inv(A_b).dot(A[:j_plus])[s] != 0:
                break_flag = True
                j_b[j_b == j_str] = j_plus
                j_b_extd = j_b_extd[j_b_extd != j_str]
        if break_flag:
            break
    if not break_flag:
        j_b[j_b == j_str] = j_0
        j_b_extd[j_b_extd == j_str] = j_0
        
print(j_b)
print(j_b_extd)
print(x_t)

[0, 1]
[0, 1, 2]
[[1. ]
 [3.5]
 [0.5]
 [0. ]]


In [272]:
while(True):
    #Шаг 1 =========================================
    c_x = np.add(c, D.dot(x_t))

    idx = np.array(j_b)
    idx_extd = np.array(j_b_extd)
    c_b = c_x[idx]
    A_b = A[:, idx]
    A_b_extd = A[:, idx_extd]
    A_b_inv = np.linalg.inv(A_b)
    u_x = A_b_inv.dot(-1*c_b)

    delta_x = np.add(np.transpose(u_x)[0].dot(A), np.transpose(c_x)[0])
    
    #Шаг 2 =========================================
    if np.all(delta_x >= 0):
        j_b = [i + 1 for i in j_b]
        j_b_extd = [i + 1 for i in j_b_extd]
        print("ОТВЕТ:")
        print("x =", np.transpose(x_t))
        print("j_b =", j_b)
        print("j_b_extd =", j_b_extd)
        break
        
    #Шаг 3 =========================================
    j_0 = np.where(delta_x < 0)[0][0]
    
    #Шаг 4 (1) =========================================
    l = np.zeros(delta_x.size)
    l[j_0] = -next(
        (item for item in delta_x if item < 0),
        None)
    
    #Шаг 4 (2) =========================================
    D_sub = D[np.ix_(j_b_extd, j_b_extd)]
    H_1 = np.concatenate((D_sub, np.transpose(A_b_extd)), axis=1)
    H_2 = np.concatenate((A_b_extd, np.zeros((A_b_extd.shape[0], H_1.shape[1] - A_b_extd.shape[1]))), axis=1)
    H = np.concatenate((H_1, H_2), axis=0)
    H_inv = np.linalg.inv(H)
 
    #Шаг 4 (3) =========================================
    d_j0 = D[:, j_0]
    b = []
    for item in j_b_extd:
        b.append(d_j0[item - 1])
    a_j0 = A[:, j_0]
    for item in a_j0:
        b.append(item)
        
    #Шаг 4 (4) =========================================        
    tmp = np.dot(-1, H_inv)
    x_hat = np.dot(tmp, b)
    
    #Шаг 4 (5) =========================================
    for i in range(len(j_b_extd)):
        l[j_b_extd[i]] = x_hat[i]
        
    #Шаг 5 (1) =========================================
    lil_delta = np.transpose(l) * D * l
    lil_delta = np.linalg.matrix_rank(lil_delta)
    
    #Шаг 5 (2) =========================================
    theta_j_0 = 0
    if lil_delta > 0:
        theta_j_0 = abs(delta_x[j_0]) / lil_delta
    elif lil_delta == 0:
        theta_j_0 = np.inf
        
    #Шаг 5 (3) =========================================
    theta_li = [theta_j_0]
    index_li = [j_0]
    for j in j_b_extd:
        if l[j] < 0:
            theta_li.append(- x_t[j] / l[j])
        elif l[j] >= 0:
            theta_li.append(np.inf)
        index_li.append(j)
        
    #Шаг 5 (4) =========================================
    theta_0_index = 0
    theta_0 = float(theta_li[0])
    for i in range(len(theta_li)):
        if theta_0 > theta_li[i]:
            theta_0 = float(theta_li[i])
            theta_0_index = i

    if (theta_0 == np.inf):
        print("Целевая функция задачи не ограничена снизу на множестве допустимых планом")
        break
    else:
        j_str = index_li[theta_0_index]
      
    #Шаг 6 (1) =========================================  
    x_t = np.transpose(np.add(np.transpose(x_t), l.dot(theta_0)))
    #Шаг 6 (2) ========================================= 
    if j_str == j_0:
        j_b_extd = np.append(j_b_extd, (j_str))
    elif (j_str in j_b_extd) and (j_str not in j_b):
        j_b_extd = np.delete(j_b_extd, j_str)
    elif j_str in j_b:
        s_list = np.where(j_b == j_str)
        j_plus_list = np.where(j_b_extd != j_b)
        break_flag = False
        for s in s_list:
            for j_plus in j_plus_list:
                if np.linalg.inv(A_b).dot(A[:j_plus])[s] != 0:
                    break_flag = True
                    j_b[j_b == j_str] = j_plus
                    j_b_extd = j_b_extd[j_b_extd != j_str]
            if break_flag:
                break
        if not break_flag:
            j_b[j_b == j_str] = j_0
            j_b_extd[j_b_extd == j_str] = j_0 

ОТВЕТ:
x = [[1.5   2.    0.    0.375]]
j_b = [1, 2]
j_b_extd = [1, 2, 4]
